In [1]:
import numpy as np
from numpy.random import normal
from numpy.random import uniform
from numpy.linalg import inv
from math import sqrt
import pandas as pd

In [2]:
N, p, sigma, low, high = 1000, 3, 1, -1, 1
b = np.array([[2, 5, -4, 0]]).T

In [3]:
sigma_squared_hat_biased = []
sigma_squared_hat_unbiased = []
for i in range(10000):
    X = uniform(low, high, size=(N, p+1))
    X[:,0] = 1
    y = X.dot(b) + normal(0, sigma, (N, 1))
    b_hat = inv(X.T.dot(X)).dot(X.T).dot(y)
    y_hat = X.dot(b_hat)
    sse = np.sum((y - y_hat)**2)
    sigma_squared_hat_biased.append(sse / N)
    sigma_squared_hat_unbiased.append(sse / (N-p-1))

In [4]:
np.mean(sigma_squared_hat_biased), np.mean(sigma_squared_hat_unbiased)

(0.9964521986605323, 1.00045401471941)

In [5]:
sigma_hat = np.sqrt(sigma_squared_hat_unbiased[-1])
V = inv(X.T.dot(X))
v = []
z = []
for j in range(p+1):
    z.append(b_hat[j][0] / sigma_hat / sqrt(V[j,j]))
    v.append(V[j,j])
v = np.array(v)

In [6]:
z

[64.5190467036647, 89.21793253268854, -73.09493254757177, -0.29192545039578793]

In [7]:
b_hat = np.squeeze(b_hat)

In [8]:
(b_hat - 1.96*np.sqrt(v)*sigma_hat), (b_hat + 1.96*np.sqrt(v)*sigma_hat)

(array([ 1.96574226,  4.78934895, -4.10871284, -0.12456314]),
 array([ 2.08891724,  5.00450702, -3.89412129,  0.09226798]))

In [15]:
data = pd.read_csv("Prostate Cancer.txt") 
data.head()

,id,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,1,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,T
1,2,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,T
2,3,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,T
3,4,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,T
4,5,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,T


In [59]:
names = ['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45']

In [60]:
data[data.train == 'T'][names].corr()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45
lcavol,1.000000,0.300232,0.286324,0.063168,0.592949,0.692043,0.426414,0.483161
lweight,0.300232,1.000000,0.316723,0.437042,0.181054,0.156829,0.023558,0.074166
age,0.286324,0.316723,1.000000,0.287346,0.128902,0.172951,0.365915,0.275806
lbph,0.063168,0.437042,0.287346,1.000000,-0.139147,-0.088535,0.032992,-0.030404
svi,0.592949,0.181054,0.128902,-0.139147,1.000000,0.671240,0.306875,0.481358
lcp,0.692043,0.156829,0.172951,-0.088535,0.671240,1.000000,0.476437,0.662533
gleason,0.426414,0.023558,0.365915,0.032992,0.306875,0.476437,1.000000,0.757056
pgg45,0.483161,0.074166,0.275806,-0.030404,0.481358,0.662533,0.757056,1.000000


In [69]:
lcavol = data[data.train == 'T'].lcavol.values
lweight = data[data.train == 'T'].lweight.values

In [70]:
lcavol_mean = np.mean(lcavol)
lweight_mean = np.mean(lweight)

lcavol_std = np.sqrt(np.mean((lcavol - lcavol_mean)**2))
lweight_std = np.sqrt(np.mean((lweight - lweight_mean)**2))
cor = np.mean((lcavol - lcavol_mean)*(lweight - lweight_mean))/(lcavol_std*lweight_std)

In [78]:
X = data[data.train == 'T'][names].values
Y = data[data.train == 'T'][['lpsa']].values

X = (X - X.mean(axis=0)) / (X.std(axis=0))

In [79]:
from sklearn import linear_model
lr = linear_model.LinearRegression().fit(X,Y)

In [80]:
lr.intercept_, lr.coef_

(array([2.45234509]),
 array([[ 0.71104059,  0.29045029, -0.14148182,  0.21041951,  0.30730025,
         -0.28684075, -0.02075686,  0.27526843]]))